## Quantify similarity/divergence of human vs synthetic

In [98]:
import sys
sys.path.append("../../../")

import numpy as np
import pandas as pd

from src.eval_utils.metric_calculator import ResultEvaluator

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Load results

In [102]:
samples = [806927261, 661238641, 817443461]
final_columns = ['ea801', 'ea802', 'ea803', 'ea804', 'ea805', 'ea806', 'ea807', 'ea808', 'ea809', 'ea810', 'ea811', 
                 'ea812', 'ea813', 'ea814', 'ea815']
opportunity_cost = [0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 900, 1200]
# Generate dataframe for evaluation
econometric_factors = ['is_east','total_income_level','city_size','is_high_wealth','high_education_level','building_period','prof_status',
                       'n_housholds','building_type','kf10']
optional_factors = ['pol_orientation','is_altruist','nature_level','ownership_level','profit_focus']
columns_to_exclude_ols = ['success', 'label', 'temperature', 'decision', 'key', 
                      'citizen_type', 'profit_focus_long', 'profit_focus_short', 
                      'ownership_level_Medium', 'ownership_level_High', 
                      'nature_level_High', 'nature_level_Medium', 
                      'pol_orientation_left', 'pol_orientation_right', 
                      'pol_orientation_center', 'is_altruist']

df_sample = pd.read_csv("../../../data/ariadne/heating_buildings/df_cata_sample_50_processed_relevant_factors_citizen_type.csv", encoding='ISO-8859-1')
df_questions = pd.read_csv("../../../data/ariadne/heating_buildings/questions_selected_survey_mapper_encoded.csv", sep=';', encoding='utf-8', on_bad_lines='skip')

### 2.Evaluate performance for temperature 0.5 in 3 samples over 3 iterations

In [ ]:
output_file = '../../../data/ariadne/heating_buildings/experiment_1/20231230_0301/recurrent_gpt-4-1106-preview-t_0.5_qa.csv'
df_synthetic_samples = pd.read_csv(output_file)

In [108]:
evaluator = ResultEvaluator(final_columns=final_columns, opportunity_costs=opportunity_cost)
df_sample_sel = evaluator.format_initial_data(df_sample, samples, econometric_factors, optional_factors)
df_synthetic_pivot = evaluator.pivot_with_opportunity_cost(df_synthetic_samples)
df_human_pivot = evaluator.pivot_with_opportunity_cost(df_sample_sel)
df_human_sample_comp = df_sample_sel.merge(df_human_pivot, on='key')
df_synthetic_sample_comp = df_sample_sel.merge(df_synthetic_pivot, on='key')
df_human_sample_trf = evaluator.prepare_for_ols_logistic_regression(df_human_sample_comp)
df_synthetic_sample_trf = evaluator.prepare_for_ols_logistic_regression(df_synthetic_sample_comp)
model_synthetic, summary_synthetic = evaluator.fit_ols_logistic_regression(df_synthetic_sample_trf, 'choice', columns_to_exclude_ols)

In [110]:
print(summary_synthetic)

                            OLS Regression Results                            
Dep. Variable:                 choice   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     16.51
Date:                Sat, 30 Dec 2023   Prob (F-statistic):           5.78e-11
Time:                        04:08:28   Log-Likelihood:                -69.620
No. Observations:                 135   AIC:                             149.2
Df Residuals:                     130   BIC:                             163.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [111]:
df_concatenated = evaluator.concatenate_and_transform_dfs(df_human_sample_trf, df_synthetic_sample_trf)
model, summary = evaluator.fit_ols_logistic_regression(df_concatenated, 'choice', columns_to_exclude_ols + ['iter'])

In [112]:
print(summary)

                            OLS Regression Results                            
Dep. Variable:                 choice   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     11.89
Date:                Sat, 30 Dec 2023   Prob (F-statistic):           1.45e-08
Time:                        04:09:35   Log-Likelihood:                -108.61
No. Observations:                 180   AIC:                             227.2
Df Residuals:                     175   BIC:                             243.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

### 3.Evaluate performance for temperature 0.7 in 3 samples over 3 iterations

In [113]:
output_file = '../../../data/ariadne/heating_buildings/experiment_1/20231230_1053/recurrent_gpt-4-1106-preview-t_0.7_qa.csv'
df_synthetic_samples = pd.read_csv(output_file)

In [116]:
evaluator = ResultEvaluator(final_columns=final_columns, opportunity_costs=opportunity_cost)
df_sample_sel = evaluator.format_initial_data(df_sample, samples, econometric_factors, optional_factors)
df_synthetic_pivot = evaluator.pivot_with_opportunity_cost(df_synthetic_samples)
df_human_pivot = evaluator.pivot_with_opportunity_cost(df_sample_sel)
df_human_sample_comp = df_sample_sel.merge(df_human_pivot, on='key')
df_synthetic_sample_comp = df_sample_sel.merge(df_synthetic_pivot, on='key')
df_human_sample_trf = evaluator.prepare_for_ols_logistic_regression(df_human_sample_comp)
df_synthetic_sample_trf = evaluator.prepare_for_ols_logistic_regression(df_synthetic_sample_comp)
model_synthetic, summary_synthetic = evaluator.fit_ols_logistic_regression(df_synthetic_sample_trf, 'choice', columns_to_exclude_ols)

In [117]:
print(summary_synthetic)

                            OLS Regression Results                            
Dep. Variable:                 choice   R-squared:                       0.404
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                     22.03
Date:                Sat, 30 Dec 2023   Prob (F-statistic):           6.71e-14
Time:                        11:02:29   Log-Likelihood:                -57.168
No. Observations:                 135   AIC:                             124.3
Df Residuals:                     130   BIC:                             138.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [118]:
df_concatenated = evaluator.concatenate_and_transform_dfs(df_human_sample_trf, df_synthetic_sample_trf)
model, summary = evaluator.fit_ols_logistic_regression(df_concatenated, 'choice', columns_to_exclude_ols + ['iter'])

In [119]:
print(summary)

                            OLS Regression Results                            
Dep. Variable:                 choice   R-squared:                       0.267
Model:                            OLS   Adj. R-squared:                  0.250
Method:                 Least Squares   F-statistic:                     15.91
Date:                Sat, 30 Dec 2023   Prob (F-statistic):           3.95e-11
Time:                        11:02:44   Log-Likelihood:                -98.618
No. Observations:                 180   AIC:                             207.2
Df Residuals:                     175   BIC:                             223.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------